In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
start = time.time()
spark = SparkSession.builder.\
            master("local").\
            appName("amzone").\
            enableHiveSupport().\
            getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('WARN')
inputPath="data/sample.json"
# rawdata.select("reviewerID","asin","reviewerName","overall","reviewTime").show()
# inputPath = "data/reviews_Books_5.json"
rawdata = spark.read.json(inputPath)
ratingDf = rawdata.select(rawdata.reviewerID.alias("user"),\
               rawdata.asin.alias("item"),\
               rawdata.overall.alias("rating"))
ratingDf = ratingDf.select(ratingDf.user,ratingDf.item,ratingDf.rating.cast(DoubleType()))
ratingDf.registerTempTable("Ratings")

sqlString="with t1 as (select user,rating from Ratings group by user,rating) select * from t1"
#           t2 as (select count(distinct(item)) as items,rating from Ratings group by rating), \
#            select * from t1 left join t2 on t1.rating=t2.rating "
sqlString2 = "select count(*) as sm,rating from Ratings group by rating"
# rawdata.select("reviewerID").distinct().count()
resultDf = spark.sql(sqlString2)
resultDf.show()

trainingData,cvData,testData = ratingDf.randomSplit([0.6,0.2,0.2])
# resultDf.show()
from pyspark.ml.pipeline import Pipeline,PipelineModel
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
userIndexer = StringIndexer(inputCol="user",outputCol="userid",handleInvalid="error").fit(ratingDf)
itemIndexer = StringIndexer(inputCol="item",outputCol="itemid").fit(ratingDf)
alsModel = ALS(rank=10,userCol="userid",itemCol="itemid",ratingCol="rating",maxIter=5,regParam=0.1)
pipeAls = Pipeline(stages=[userIndexer,itemIndexer,alsModel])
pipeAlsModel = pipeAls.fit(trainingData)
predictionAls = pipeAlsModel.transform(trainingData)
predictionRecommendation= pipeAlsModel.transform(testData)
# print predictionRecommendation.select(isnull('prediction').alias('r')).filter('r=true').count()
evaluator = RegressionEvaluator()\
      .setMetricName("rmse")\
      .setLabelCol("rating")\
      .setPredictionCol("prediction")
rmse = evaluator.evaluate(predictionRecommendation)
predictionAls.sort(asc("userid")).limit(10).show()
predictionRecommendation.sort(asc("userid")).limit(10).show()
#print "RMSE:",rmse
end = time.time()
print "total:%f s" %(end-start)

+---+------+
| sm|rating|
+---+------+
|  1|   1.0|
|  4|   4.0|
|  3|   3.0|
|  3|   2.0|
| 89|   5.0|
+---+------+

+--------------+----------+------+------+------+----------+
|          user|      item|rating|userid|itemid|prediction|
+--------------+----------+------+------+------+----------+
| A1FQPOYRBTTK1|000100039X|   5.0|   0.0|   0.0| 4.9524026|
|A3FFNE1DR5SI1W|000100039X|   5.0|   1.0|   0.0| 4.9524026|
|A1NPNGWBVD9AK3|000100039X|   5.0|   3.0|   0.0| 4.9524026|
|  A9WX8DK93SN5|000100039X|   1.0|   4.0|   0.0| 0.9904805|
| AMRZ5G7HF7I03|000100039X|   5.0|   9.0|   0.0| 4.9524026|
|A19N3FCQCLJYUA|000100039X|   5.0|  10.0|   0.0| 4.9524026|
|A1TT4CY55WLHAR|000100039X|   5.0|  12.0|   0.0| 4.9524026|
|A3FI0744PG1WYG|000100039X|   5.0|  14.0|   0.0| 4.9524026|
| AAFLZI7MX9UIG|000100039X|   5.0|  15.0|   0.0| 4.9524026|
|A2I35JB67U20C0|000100039X|   5.0|  17.0|   0.0| 4.9524026|
+--------------+----------+------+------+------+----------+

+--------------+----------+------+------+

In [12]:
# predictionRecommendation.count()
# predictionAls.
rmse = evaluator.evaluate(predictionRecommendation.na.fill(9))
predictionRecommendation.na.fill(9).filter('prediction=9').collect()
print rmse

Py4JJavaError: An error occurred while calling o582.showString.
: org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.ScalaUDF.eval(ScalaUDF.scala:1072)
	at org.apache.spark.sql.catalyst.expressions.EqualNullSafe.eval(predicates.scala:470)
	at org.apache.spark.sql.catalyst.optimizer.EliminateOuterJoin$.org$apache$spark$sql$catalyst$optimizer$EliminateOuterJoin$$canFilterOutNull(joins.scala:116)
	at org.apache.spark.sql.catalyst.optimizer.EliminateOuterJoin$$anonfun$7.apply(joins.scala:125)
	at org.apache.spark.sql.catalyst.optimizer.EliminateOuterJoin$$anonfun$7.apply(joins.scala:125)
	at scala.collection.LinearSeqOptimized$class.exists(LinearSeqOptimized.scala:93)
	at scala.collection.immutable.List.exists(List.scala:84)
	at org.apache.spark.sql.catalyst.optimizer.EliminateOuterJoin$.org$apache$spark$sql$catalyst$optimizer$EliminateOuterJoin$$buildNewJoinType(joins.scala:125)
	at org.apa

4.35252161907
